In [ ]:
from mini_latent_pd.models.minifold.model.model import MiniFoldModel

In [1]:
import tree

In [1]:
import torch
import torch.nn.functional as F
from pathlib import Path
from mini_latent_pd.models.minifold.model.model import MiniFoldModel
from mini_latent_pd.models.minifold.data.config import model_config
from mini_latent_pd.models.minifold.data.of_data import of_inference
from mini_latent_pd.models.minifold.utils.residue_constants import restype_order_with_x_inverse
from esm.pretrained import load_model_and_alphabet

def load_minifold_model(cache_dir="./minifold_cache", model_size="48L", device="cuda"):
    """
    Loads the MiniFold model, alphabet, and config for interactive use.
    """
    # 1. Paths
    cache = Path(cache_dir).expanduser()
    checkpoint_path = cache / f"minifold_{model_size}.ckpt"
    
    if not checkpoint_path.exists():
        raise FileNotFoundError(f"Checkpoint not found at {checkpoint_path}. Run predict.py once to download it.")

    print(f"Loading checkpoint from {checkpoint_path}...")
    
    # 2. Load Checkpoint & Hyperparams
    ckpt = torch.load(checkpoint_path, map_location="cpu")
    hparams = ckpt["hyper_parameters"]
    
    # 3. Initialize Model
    # We create the config required for feature generation
    config_of = model_config(
        "initial_training",
        train=False,
        low_prec=False,
        long_sequence_inference=False,
    )
    
    model = MiniFoldModel(
        esm_model_name=hparams["esm_model_name"],
        num_blocks=hparams["num_blocks"],
        no_bins=hparams["no_bins"],
        config_of=config_of,
        use_structure_module=True,
        kernels=False, # Important: False for Mac/MPS compatibility
    )

    # 4. Load State Dict
    state_dict = ckpt["state_dict"]
    # Clean keys as done in predict.py
    state_dict = {k: v for k, v in state_dict.items() if "boundaries" not in k}
    state_dict = {k: v for k, v in state_dict.items() if "mid_points" not in k}
    state_dict = {k.replace("_orig_mod.", ""): v for k, v in state_dict.items()}
    state_dict = {k.replace("model.", ""): v for k, v in state_dict.items()}
    
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    
    # 5. Load Alphabet (needed for tokenizing sequences)
    _, alphabet = load_model_and_alphabet(hparams["esm_model_name"])
    
    return model, alphabet, config_of.data

In [2]:
class MinifoldCollator:
    def __init__(self, alphabet, config):
        self.alphabet = alphabet
        self.config = config

    def __call__(self, batch):
        """
        Transforms a list of mdCATH dicts into a MiniFold batch.
        """
        seq_tokens_list = []
        masks_list = []
        batch_of_list = []
        gt_coords_list = []
        ids = []

        for item in batch:
            seq_str = item["sequence"]
            ids.append(item["id"])
            
            # --- A. Generate Features ---
            # This replicates predict.py's `prepare_input`
            of_feats = of_inference(seq_str, "predict", self.config)
            
            # Reconstruct clean sequence from features to ensure alignment
            clean_seq = "".join(
                [restype_order_with_x_inverse[x.item()] for x in of_feats["aatype"]]
            )[: of_feats["seq_length"]]
            
            # Encode for ESM
            encoded_seq = torch.tensor(self.alphabet.encode(clean_seq), dtype=torch.long)
            
            # --- B. Collect ---
            seq_tokens_list.append(encoded_seq)
            masks_list.append(of_feats["seq_mask"][:, 0].bool())
            
            # Filter OpenFold features to only what MiniFold needs
            relevant = {"aatype", "seq_mask", "residx_atom37_to_atom14", "atom37_atom_exists"}
            batch_of_list.append({k: v for k, v in of_feats.items() if k in relevant})
            
            # Collect Ground Truth Coords (from mdCATH)
            gt_coords_list.append(torch.tensor(item["coords"]))

        # --- C. Padding ---
        max_len = max(len(s) for s in seq_tokens_list)
        
        # 1. Pad ESM Tokens (1 is usually padding in ESM alphabet)
        padded_seqs = torch.stack([
            F.pad(s, (0, max_len - len(s)), value=1) for s in seq_tokens_list
        ])
        
        # 2. Pad Masks
        padded_masks = torch.stack([
            F.pad(m, (0, max_len - len(m)), value=0) for m in masks_list
        ])
        
        # 3. Pad OpenFold Features
        batched_of = {}
        if batch_of_list:
            for k in batch_of_list[0].keys():
                feats = [d[k] for d in batch_of_list]
                # Pad the first dimension (sequence length)
                batched_of[k] = torch.stack([
                    F.pad(f, [0] * 2 * (len(f.shape) - 1) + [0, max_len - f.shape[0]], value=0) 
                    for f in feats
                ])

        # 4. Pad Ground Truth Coords
        # shape: (Batch, Max_Len, 3)
        padded_coords = torch.nn.utils.rnn.pad_sequence(
            gt_coords_list, batch_first=True, padding_value=0.0
        )

        return {
            "id": ids,
            "seq": padded_seqs,
            "mask": padded_masks,
            "batch_of": batched_of,
            "gt_coords": padded_coords
        }

In [3]:
import torch
from datasets import IterableDataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# Import the generator from your new file
# (Ensure mdcath_dataset.py is in the same folder as your notebook)
from mini_latent_pd.data.mdcath_dataset import mdcath_generator

# --- Configuration ---
REPO_ID = "compsciencelab/mdCATH"
SUB_SAMPLE_PROTEINS = 4 
BATCH_SIZE = 1

# --- Padding Collator (Keep this in notebook as it's simple) ---
def padding_collate(batch):
    coords_list = [torch.tensor(item["coords"]) for item in batch]
    coords_padded = pad_sequence(coords_list, batch_first=True, padding_value=0.0)
    lengths = torch.tensor([len(c) for c in coords_list])
    B, Max_L = coords_padded.shape[:2]
    mask = torch.arange(Max_L).expand(B, Max_L) < lengths.unsqueeze(1)

    return {
        "id": [item["id"] for item in batch],
        "coords": coords_padded,
        "mask": mask,
        "temp": torch.tensor([item["temp"] for item in batch]),
        "lengths": lengths
    }

# --- Execution ---

if __name__ == "__main__":
    print("--- 1. Initializing Streaming Dataset ---")
    
    # Now this works because mdcath_generator is an imported function
    ds = IterableDataset.from_generator(
        mdcath_generator, 
        gen_kwargs={"repo_id": REPO_ID, "sub_sample_proteins": SUB_SAMPLE_PROTEINS}
    )
    
    # Optional: Hugging Face dataset mapping
    ds = ds.map(lambda x: {"num_atoms": len(x["coords"])})

    print("--- 2. Running DataLoader with Padding ---")
    dataloader = DataLoader(ds, batch_size=BATCH_SIZE, collate_fn=padding_collate)

    for i, batch in enumerate(dataloader):
        print(f"\nBatch {i+1}")
        print(f"  IDs: {batch['id']}")
        print(f"  Coords Shape: {batch['coords'].shape} (Padded)")
        
        if i >= 4: 
            print("Test complete.")
            break

/Users/moustholmes/mini_latent_pd/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 1. Initializing Streaming Dataset ---
--- 2. Running DataLoader with Padding ---
Generator finding files... (using subset of 4)

Batch 1
  IDs: ['2cw7A03']
  Coords Shape: torch.Size([1, 2769, 3]) (Padded)

Batch 2
  IDs: ['2cw7A03']
  Coords Shape: torch.Size([1, 2769, 3]) (Padded)

Batch 3
  IDs: ['2cw7A03']
  Coords Shape: torch.Size([1, 2769, 3]) (Padded)

Batch 4
  IDs: ['2cw7A03']
  Coords Shape: torch.Size([1, 2769, 3]) (Padded)

Batch 5
  IDs: ['2cw7A03']
  Coords Shape: torch.Size([1, 2769, 3]) (Padded)
Test complete.


In [1]:
import urllib.request
from pathlib import Path

# Config
cache_dir = Path("./minifold_cache")
model_size = "12L" # or "48L" if you want the larger one
filename = f"minifold_{model_size}.ckpt"
target_path = cache_dir / filename

# Create dir
cache_dir.mkdir(parents=True, exist_ok=True)

# Download if missing
if not target_path.exists():
    print(f"Downloading {filename}...")
    url = f"https://huggingface.co/jwohlwend/minifold/resolve/main/minifold_{model_size}_final.ckpt"
    urllib.request.urlretrieve(url, str(target_path))
    print("Download complete.")
else:
    print(f"Found checkpoint at {target_path}")

Download complete.


In [4]:
def crop_batch(batch, max_len=600):
    """
    Hard-crops a batch to a maximum length to prevent OOM/Buffer errors.
    """
    current_len = batch["seq"].shape[1]
    if current_len <= max_len:
        return batch
    
    print(f"⚠️ Cropping batch from {current_len} to {max_len} for testing...")
    
    # Crop sequence and mask
    new_batch = {
        "seq": batch["seq"][:, :max_len],
        "mask": batch["mask"][:, :max_len],
        "batch_of": {},
        "id": batch["id"] # Keep IDs
    }
    
    # Crop OpenFold features (careful with dimensions)
    for k, v in batch["batch_of"].items():
        if v.shape[1] == current_len:
             # Assuming dim 1 is sequence length (Batch, Len, ...)
            new_batch["batch_of"][k] = v[:, :max_len, ...]
        else:
            # Some features might not map directly to len (check specific keys if needed)
            new_batch["batch_of"][k] = v
            
    return new_batch

In [6]:
from mini_latent_pd.data.mdcath_dataset import mdcath_generator

# 1. Setup Device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load Model
# Ensure you point cache_dir to where predict.py downloaded weights
model, alphabet, config = load_minifold_model(cache_dir="./minifold_cache", model_size="12L", device=device)

# 3. Prepare Dataset
# (Assuming mdcath_generator is defined in a previous cell)
from torch.utils.data import DataLoader
from datasets import IterableDataset

ds = IterableDataset.from_generator(
        mdcath_generator, 
        gen_kwargs={"repo_id": REPO_ID, "sub_sample_proteins": SUB_SAMPLE_PROTEINS}
    )
collator = MinifoldCollator(alphabet, config)

# 4. Create DataLoader
loader = DataLoader(ds, batch_size=2, collate_fn=collator)

# 5. Run a Test Batch
print("Fetching batch...")
batch = next(iter(loader))

# Move to device
batch_gpu = {
    "seq": batch["seq"].to(device),
    "mask": batch["mask"].to(device),
    "batch_of": {k: v.to(device) for k, v in batch["batch_of"].items()}
}

# batch_gpu = crop_batch(batch_gpu, max_len=512)

print(f"Running inference for protein IDs: {batch['id']}")
with torch.no_grad():
    # Run MiniFold!
    output = model(batch_gpu)

print("\nSuccess!")
print("Latent Pair Representation shape:", output["pair"].shape) # (B, L, L, 128)
print("Predicted Coordinates shape:", output["final_atom_positions"].shape) # (B, L, 37, 3)

Using device: mps
Loading checkpoint from minifold_cache/minifold_12L.ckpt...
Fetching batch...
Generator finding files... (using subset of 4)
Running inference for protein IDs: ['1l1cA00', '1l1cA00']

Success!
Latent Pair Representation shape: torch.Size([2, 883, 883, 128])
Predicted Coordinates shape: torch.Size([2, 883, 37, 3])


/Users/moustholmes/mini_latent_pd/src/mini_latent_pd/models/minifold/utils/tensor_utils.py:86: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/python_variable_indexing.cpp:312.)
  return data[ranges]


In [8]:
for key, value in output.items():
    print(f"{key}: {value.shape if isinstance(value, torch.Tensor) else type(value)}")

preds: torch.Size([2, 883, 883, 64])
pair: torch.Size([2, 883, 883, 128])
single: torch.Size([2, 883, 1024])
sm: <class 'dict'>
final_atom_positions: torch.Size([2, 883, 37, 3])
final_atom_mask: torch.Size([2, 883, 37])
final_affine_tensor: torch.Size([2, 883, 4, 4])
lddt_logits: torch.Size([2, 883, 50])
plddt: torch.Size([2, 883])


In [10]:
for key, value in batch.items():
    print(f"{key}: {value.shape if isinstance(value, torch.Tensor) else type(value)}")

id: <class 'list'>
seq: torch.Size([2, 883])
mask: torch.Size([2, 883])
batch_of: <class 'dict'>
gt_coords: torch.Size([2, 883, 3])
